### 本文内容主要来自：
- http://blog.fatedier.com/2014/09/08/learn-to-write-makefile-01/
- http://www.ruanyifeng.com/blog/2015/02/make.html

### 概述
而Makefile就相当于是一个规则文件，make程序会按照Makefile所指定的规则，去判断哪些文件需要先编译，哪些文件需要后编译，哪些文件需要重新编译。

### makefile基本规则
makefile的基本规则是有以下构成：<br>
TARGET... :PREREQUISITES...<br>
$\ \ \ $    COMMAND<br>
$\ \ \ $    ...<br>
$\ \ \ $    ...<br>
- TARGET：规则生成的目标文件，通常是需要生成的程序名（目标文件）或者过程文件（类似.o文件）。
- PREREQUISITES：规则的依赖项，比如前面的Makefile文件中我们生成test程序所依赖的就是test.cpp。
- COMMAND：规则所需执行的命令行，通常是编译命令。这里需要注意的是每一行命令都需要以[TAB]字符开头。

这个规则，用通俗的自然语言翻译过来就是：
- 如果目标test文件不存在，根据规则创建它。
- 目标test文件存在，并且test文件的依赖项中存在任何一个比目标文件更新（比如修改了一个函数，文件被更新了），根据规则重新生成它。
- 目标test文件存在，并且它比所有的依赖项都更新，那么什么都不做。

### 多个文件的编译
一个项目不可能只有一个文件，学会了单个文件的编译，自然而然就要考虑如何去编译多个文件呢？<br>
同样，假设当前目录下有如下7个文件，test.cpp、w1.h、w1.cpp、w2.h、w2.cpp、w3.h、w3.cpp。其中test.cpp包含main函数，并且引用了w1.h、w2.h以及w3.h。我们需要生成的程序名为test。<br>
Makefile具体内容如下:

In [ ]:
test:test.cppw1.o w2.o w3.o
    g++ -o test test.cpp w1.o w2.o w3.o
w1.o:w1.cpp
    g++ -c -o w1.o w1.cpp
w2.o:w2.cpp
    g++ -c -o w2.o w2.cpp
w3.o:w3.cpp
    g++ -c -o w3.o w3.cpp

这里需要注意的是，我们写的第一个规则的目标，将会成为“终极目标”，也就是我们最终希望生成的程序，这里是“test”文件。根据我们的“终极目标”，make会进行自动推导，例如“终极目标”依赖于的.o文件，make就会寻找生成这些.o文件的规则，然后执行相应的命令去生成这些文件，这样一层一层递归地进行下去，直到最终生成了“终极目标”。<br>
### 使用伪目标来清除过程文件
clean:<br>
$\ \ \ $    -rm–f test *.o<br>

### 使用变量来简化makefile
作为一个“懒惰”的程序员，现在问题又来了。如果按照上面的写法，在文件数量和名称不变的情况的下确实是没有问题，但是如果我们新增一个文件的话，岂不是又要去修改Makefile了，一个项目多的可能有成百上千的文件，这样管理起来得有多麻烦呀！<br>
我们在Linux下如果要查看当前目录下所有的cpp文件的时候，使用的命令ls *.cpp<br>
通过这个命令，我们就可以将所有的cpp文件名称显示在界面上。而在Makefile中我们同样可以使用类似的规则来做简化，进一步减少后续开发过程中对Makefile文件的修改。<br>
修改后的Makefile文件如下：

In [ ]:
TARGET = test

CPP_FILES = $(shell ls *.cpp)
BASE = $(basename $(CPP_FILES))
OBJS = $(addsuffix .o, $(addprefix obj/,$(BASE)))
 
$(TARGET):$(OBJS)
    -rm -f $@
    g++ -o $(TARGET)$(OBJS)
 
obj/%.o:%.cpp
    @if test ! -d"obj"; then\
    mkdir -pobj;\
    fi;
    g++ -c -o $@ $<
 
clean:
    -rm -f test
    -rm -f obj/*.o

##### CPP_FILES = \$(shell ls *.cpp)
定义一个变量，内容为所有的以.cpp为后缀的文件的文件名，以空格隔开。<br>
这里&(shell 命令)的格式，说明这里将会用shell命令执行后输出的内容进行替换，就和在命令行下输入ls *.cpp得到的结果一样。

##### BASE = \$(basename \$(CPP_FILES))
定义一个变量，内容为所有的以.cpp为后缀的文件的文件名去除掉后缀部分。<br><br>
$(CPP_FILES)是引用CPP_FIFES这个变量的内容，相信学过如何写shell命令的同学肯定不会陌生。basename 是一个函数，其作用就是去除掉文件名的后缀部分，例如“test.cpp”，经过这一步后就变成了“test”<br>

##### OBJS = \$(addsuffix .o, \$(addprefix obj/,\$(BASE)))
定义一个变量，内容为所有的以.cpp为后缀的文件去除调后缀部分后加上“.o”。<br>
和basename一样，addsuffix和addprefix同样也是调用函数。addprefix的作用是给每个文件名加上前缀，这里是加上“obj/”，而addsuffix的作用是给每个文件名加上后缀，这里是在文件名后加上“.o”。例如“test”，经过变换后变成了“obj/test.o”。<br>

##### obj/%.o:%.cpp
下面命令行的前三行，具体的作用是检查当前目录下是否有名为“obj”的目录，如果没有，则使用mkdir命令创建这个目录。如果不了解的同学不如先去看一下shell编程的相关知识吧。<br>
最后一句中的\$@前面已经解释过了，是代表规则的目标文件名称，而\$<与之对应的，则是代表规则的依赖项中第一个依赖文件的名称。<br>
例如obj/test.o:test.cpp<br>
那么\$@的值为“test.o”，\$<的值为“test.cpp”

### 补充
COMMAND需要注意的是，每行命令在一个单独的shell中执行。这些Shell之间没有继承关系。<br>
var-lost:<br>
$\ \ \ $    export foo=bar<br>
$\ \ \ $    echo "foo=[\$\$foo]"<br>
上面代码执行后（make var-lost），取不到foo的值。因为两行命令在两个不同的进程执行。一个解决办法是将两行命令写在一行，中间用分号分隔。<br>
var-kept:<br>
$\ \ \ $    export foo=bar; echo "foo=[\$\$foo]"<br>
另一个解决办法是在换行符前加反斜杠转义。<br>
var-kept:<br>
$\ \ \ $    export foo=bar; \<br>
$\ \ \ $    echo "foo=[\$\$foo]"<br>
最后一个方法是加上.ONESHELL:命令
.ONESHELL:<br>
var-kept:<br>
$\ \ \ $    export foo=bar; <br>
$\ \ \ $    echo "foo=[\$\$foo]"<br>


### Makefile文件中的语法
##### 注释
井号（#）在Makefile中表示注释。
##### 回声（echoing）
正常情况下，make会打印每条命令，然后再执行，这就叫做回声（echoing）。在命令的前面加上@，就可以关闭回声。
##### 通配符
通配符（wildcard）用来指定一组符合条件的文件名。Makefile 的通配符与 Bash 一致，主要有星号（*）、问号（？）和 [...] 。比如， *.o 表示所有后缀名为o的文件
##### 模式匹配
Make命令允许对文件名，进行类似正则运算的匹配，主要用到的匹配符是%。比如，假定当前目录下有 f1.c 和 f2.c 两个源码文件，需要将它们编译为对应的对象文件。<br>
%.o:%.c
##### 变量和赋值符
Makefile 允许使用等号自定义变量
txt = Hello World<br>
test:<br>
$\ \ \ $    @echo \$(txt)<br>
<br>
上面代码中，变量 txt 等于 Hello World。调用时，变量需要放在 \$( ) 之中。<br>
调用Shell变量，需要在美元符号前，再加一个美元符号，这是因为Make命令会对美元符号转义。<br>
test:<br>
$\ \ \ $    @echo $$HOME<br>
<br>
有时，变量的值可能指向另一个变量。<br>
v1 = \$(v2)<br>
<br>
上面代码中，变量 v1 的值是另一个变量 v2。这时会产生一个问题，v1 的值到底在定义时扩展（静态扩展），还是在运行时扩展（动态扩展）？如果 v2 的值是动态的，这两种扩展方式的结果可能会差异很大。<br>
为了解决类似问题，Makefile一共提供了四个赋值运算符 （=、:=、？=、+=）<br>
VARIABLE = value<br>
\# 在执行时扩展，允许递归扩展。<br>
VARIABLE := value<br>
\# 在定义时扩展。<br>
VARIABLE ?= value<br>
\# 只有在该变量为空时才设置值。<br>
VARIABLE += value<br>
\# 将值追加到变量的尾端。<br>

##### 内置变量 (Implicit Variables)
Make命令提供一系列内置变量，比如，$(CC) 指向当前使用的编译器，$(MAKE) 指向当前使用的Make工具。这主要是为了跨平台的兼容性，
##### 自动变量（Automatic Variables）
Make命令还提供一些自动变量，它们的值与当前规则有关。主要有以下几个
- \$@ 指代当前目标，就是Make命令当前构建的那个目标。

a.txt b.txt: <br>
$\ \ \ $    touch \$@<br>
<br>
等同于下面的写法。<br>
a.txt:<br>
$\ \ \ $    touch a.txt<br>
b.txt:<br>
$\ \ \ $    touch b.txt<br>

- \$< 指代第一个前置条件。比如，规则为 t: p1 p2，那么\$< 就指代p1
- \$? 指代比目标更新的所有前置条件，之间以空格分隔。比如，规则为 t: p1 p2，其中 p2 的时间戳比 t 新，\$?就指代p2
- \$^ 指代所有前置条件，之间以空格分隔。比如，规则为 t: p1 p2，那么 \$^ 就指代 p1 p2
- \$\* 指代匹配符 % 匹配的部分， 比如% 匹配 f1.txt 中的f1 ，\$\* 就表示 f1
- \$(@D) 和 \$(@F) 分别指向 \$@ 的目录名和文件名。比如，\$@是 src/input.c，那么\$(@D) 的值为 src ，\$(@F) 的值为 input.c
- \$(<D) 和 \$(<F) 分别指向 \$< 的目录名和文件名。

##### 判断和循环
Makefile使用 Bash 语法，完成判断和循环<br>
ifeq (\$(CC),gcc)<br>
$\ \ \ $  libs=\$(libs_for_gcc)<br>
else<br>
$\ \ \ $  libs=\$(normal_libs)<br>
endif<br>
<br>
LIST = one two three<br>
all:<br>
$\ \ \ $    for i in \$(LIST); do \<br>
$\ \ \ $$\ \ \ $        echo \$\$i; \<br>
$\ \ \ $    done<br>

##### 函数
Makefile 还可以使用函数，格式如下。<br>
$(function arguments)<br>
\# 或者<br>
\${function arguments}<br>
<br>
Makefile提供了许多内置函数，可供调用。下面是几个常用的内置函数
- shell 函数用来执行 shell 命令

srcfiles := \$(shell echo src/{00..99}.txt)
- wildcard 函数用来在 Makefile 中，替换 Bash 的通配符

srcfiles := \$(wildcard src/*.txt)
- subst 函数用来文本替换，格式如下。

\$(subst from,to,text)
- patsubst 函数用于模式匹配的替换，格式如下

\$(patsubst pattern,replacement,text)
- 替换后缀名函数的写法是：变量名 + 冒号 + 后缀名替换规则。它实际上patsubst函数的一种简写形式。

上面代码的意思是，将变量OUTPUT中的后缀名 .js 全部替换成 .min.js 